# Clase 5 - Feature Engineering 📎

- MDS7202: Laboratorio de Programación Científica para Ciencia de Datos

## Objetivos de la Clase 🎯

- Entender qué es el feature engineering y cómo puede impactar en el performance de los modelos
- Aprender a implementar técnicas de feature engineering a través de Scikit-learn usando `Pipeline` y `ColumnTransformer`
- Entender y resolver la complejidad de datos faltantes




## ¿Qué es Feature Engineering? 🧮



Feature engineering es el proceso de seleccionar, transformar y crear características relevantes de entrada para construir un modelo de aprendizaje automático preciso y eficiente.

> **Pregunta ❓:** ¿Se necesita algún conocimiento previo para realizar Feature Engineering?

## Problema a visitar: House Pricing

![House Pricing](https://www.kaggle.com/competitions/5407/images/header)

Fuente: https://www.kaggle.com/c/house-prices-advanced-regression-techniques

Al igual que en la clase anterior, el dataset **`house pricing`** consiste en 80 variables (79 variables explicativas más una variable objetivo) que describen aspectos fundamentales de hogares residenciales en la ciudad de *Ames, Iowa*.

La variable objetivo es el precio final de cada hogar (regresión)

In [ ]:
# Importamos librerías a utilizar en la clase
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import norm

import plotly.express as px

In [ ]:
# El conjunto a trabajar es el de entrenamiento
df = pd.read_csv(
    "https://raw.githubusercontent.com/MDS7202/MDS7202/main/recursos/2023-01/13-EDA//train.csv",
    index_col="Id",
)

> **Actividad 📎**: Imaginen que nos pasan estos datos y nos señalan que el conjunto de datos esta compuesto por:

- Terreno 🏔️
        "LotArea",  # Area del terreno
        "LandSlope",  # Pendiente del terreno
        "Neighborhood",  # Barrio
- Metadatos de la Vivienda 📆
        "BldgType",  # Tipo de vivienda
        "YearBuilt",  # Año de construcción
        "YearRemodAdd",  # Año de remodelación
        "Utilities",  # Agua, luz, etc...
- Materiales 🧱
        "Foundation",  # Fundación de la vivienda
        "RoofMatl",  # Material del techo
        "RoofStyle",  # Estilo del techo
        "Exterior1st",  # Material del Exterior
        "ExterCond",  # Condición del material exterior
- Interior de la casa 🏡
        "GrLivArea",  # Area habitable sobre el nivel del suelo.
        "1stFlrSF",  # Area primer piso
        "2ndFlrSF",  # Area segundo piso
        "FullBath",  # Baños completos
        "HalfBath",  # Baños de visita?
        "BedroomAbvGr",  # Piezas
        "KitchenAbvGr",  # Cocinas
        "KitchenQual",  # Calidad de la cocina
- Sótano 🪨
        "TotalBsmtSF",  # Total sótano
        "BsmtCond",  # Condición del sótano
- Garaje 🚗
        "GarageType",  # Tipo de garaje
        "GarageCars",  # Cantidad de autos por garaje
- Piscina 🤽‍♂️
        "PoolArea",  # Area de la piscina
        "PoolQC",  # Calidad de la piscina
- Calefacción y Aire 🌦️
        "Heating",  # Calefacción
        "HeatingQC",  # Calidad de la Calefacción
        "CentralAir",  # Aire Acondicionado Central
- Calidad y Condición 🌟
        "OverallQual",  # Calidad general
        "OverallCond",  # Condición general actual
- Datos de la venta  💵
        "SaleType",  # Tipo de venta
        "SaleCondition",  # Condición de la vivienda en la venta
        "SalePrice",  # Precio de la venta

In [ ]:
df.head(5)

Qué deberíamos hacer ahora?

### ¿Por qué es importante el Feature Engineering? 🤨

Muchos de los algoritmos de aprendizaje automatico **poseen un mejor desempeño cuando los valores de las features** (aka columnas) **se transforman a un valor facil de interpretar por los modelos**. Por otro lado, datos sucios pueden entorpecer las predicciones generadas por nuestros modelos, al igual que las escalas en que se presentan los datos. Por esto, **es relevante que las features que utilizemos se encuentren en escalas similares y con distribuciones relativamente similares a la distribución normal**.

> Importante ❗: Gran parte de los modelos que se generan en la industria son centrados en datos, a diferencia de la academia el trabajo no se centra en el desarrollo de modelos ultra complejos, sino en modelos completamente estandarizados y por ello se necesita un mayor tiempo en la extracción de features desde los datos. Fuente interesante: [Data-Centric Approach vs Model-Centric Approach in Machine Learning](https://neptune.ai/blog/data-centric-vs-model-centric-machine-learning).

Consideren los siguientes ejemplos:

**Ejemplo 1**

- El gráfico de la izquierda se grafica una variable con respecto a otra sin escalar
- El gráfico de la derecha muestra ambas variables estandarizadas

> **Pregunta ❓:** ¿Es entendible para un humano el gráfico de la derecha?¿Qué se puede interpretar de este?¿Qué efectos tendrá usar las variables no escaladas (izquierda) vs las escaladas (derecha) usando algún modelo predictivo basado en distancias?

<img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/14-Feature-Engineering-Parte-I/escalamiento_2.png?raw=true' width=1000 />

<div align='center'>
    Fuente: <a href='https://scikit-learn.org/stable/auto_examples/preprocessing/plot_all_scaling.html'>Compare the effect of different scalers on data with outliers en el User Guide de Scikit-learn</a>.
</div>

<div align='center'>
    Los Datos son del dataset <a href='https://scikit-learn.org/stable/datasets/real_world.html#california-housing-dataset'>California Housing dataset</a>.
</div>

**Ejemplo 2**

- El gráfico de la derecha se grafica una variable un conjunto de variables en 2 dimensiones diferenciadas por la clase.
- El gráfico de la izquierda se grafica el mismo grafico pero con el conjunto de variables transformado

> **Pregunta ❓:** ¿Cual de los dos conjuntos de datos es mas facil de separar?¿Que efectos tendría utilizar las variables en el estado original contra el estado final para algún modelo lineal?.

<img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/14-Feature-Engineering-Parte-I/feature-engineering1.jpg?raw=true' width=1000 />

<div align='center'>
    Fuente: <a href='https://www.kdnuggets.com/2018/12/feature-engineering-explained.html'>Feature Engineering for Machine Learning: 10 Examples</a>.
</div>

### ¿Pero que hay del Deep Learning, no que solucionaba este problema? 😣

Si bien uno de los aspectos más relevantes del Deep Learning es la extracción automática de las features desde los datos, sin embargo:

- El desarrollo actual de deep learning no nos permite abstraer todas las features desde los datos.
- Muchas empresas no utilizan deep learning para generar modelos, ya sea por capacidad o simplemente porque no lo necesitan. **Recalcar, el deep learning no es la solución para todo!** [Ejemplo](https://arxiv.org/pdf/2106.03253.pdf)
- Deep learning suele tener buenos resultados en datos no estructurados, sin embargo, en datos tabulares no siempre es el caso.

## Veamos nuestra librería core: Scikit-Learn ⚒️

<img src="https://raw.githubusercontent.com/MDS7202/MDS7202/a60ef458182c2f26af3aaf4f8e0446a8512d4f75/clases/2022-01/15_Preprocesamiento_Intro_a_Scikit-Learn/resources/scikit-learn.png" width=400/>

[`scikit-learn`](https://scikit-learn.org/stable/) es probablemente una de las librerías de Aprendizaje Automático más populares para Python. *Open-source* y construida sobre `numpy`, `scipy` y `matplotlib`, ofrece interfaces y flujos de trabajos (*frameworks*) simples y eficientes para construir aplicaciones enfocadas análisis de datos y predicción.
Sus *APIs* permite generar código limpio y está provista de una extensa documentación.

> Parentesis: API: **Application Programming Interface / Interfaz de programación de aplicaciones** - Son las interfaces comunes (funciones, objetos, métodos, etc..) que un software o librería ofrece para comunicarse con el resto. Esta define entre otras cosas: el tipo de llamadas o funciones que pueden ser hechas, como hacerlas, el tipo de datos de entrada y salida, etc...

Una gran ventaja de `Scikit-learn` consiste en su estructura transversal de clases y herencia. La mayoría de clase pertenece a alguna de estas dos categorías:

* **`transformers`**: Permite **transformar datos input antes de utilizar algoritmos de aprendizaje sobre ellos**. A partir de este módulo, se puede realizar imputaciones de valores faltantes, estandarización de variables, escalamientos y seleccion de caracterísiticas por medio de algoritmos especializados. Esto comunmente se logra a través de las interfaces
    - `fit` que permite aprender los parámetros de la transformación, por ejemplo la media y varianza en la normalización.
    - `transform` que aplica la transformación a los datos.
    - `fit_transform` permite ambas operaciones al mismo tiempo.

* **`estimators`**: Proveen los **algoritmos** de aprendizaje automático a través de los métodos `fit` y `predict`.

El método usual de importación se basa en seleccionar un submódulo de la librería indicando (de manera opcional) el objeto que se utilizará. Por ejemplo, si se desea utilizar el escalador de datos Min-Máx del submódulo `preprocessing`, se haría de la manera usual, por medio de:

```python
from sklearn.preprocessing import StandardScaler
```

> **Nota**: No se recomienda importar la librería completa `import sklearn as sk` pues su estructura de submódulos es suficientemente grande, como para considerar cada uno como una librería.


A lo largo del curso se estudiarán distintos componentes de esta librería. Durante esta clase nos centraremos en los módulos `preprocessing`, `compose` y `pipeline`.

## Operaciones Comunes de Feature Engineering

Debido a la importancia que posee de la etapa de Feature Engineering en los proyectos de ML, se han desarrollado muchas técnicas para agilizar el proceso.

Algunas de las operaciones que podemos realizar con el módulo **`transformers`** incluyen:

- **Creación de nuevas Features** a partir de operaciones usando los datos disponibles.
- **Transformaciones** como las vistas en la clase de preprocesamiento (escalamiento, normalización, one hot encoding para variables categóricas etc...).
- **Reducción de Características** en la que se combinan/reducen características redundantes (usando por ejemplo, PCA).
- **Selección de Características** en la que a partir de diversos criterios se seleccionan las características que más aportan al modelo.

El proceso de generar y preprocesar las features requiere mucha creatividad y al mismo conocimiento del dominio del problema.

Para efectos de esta clase revisaremos las siguientes:

- **Escalamiento de variables**
- **Codificación de características categóricas**
- **Missing Values**

Sin embargo, a pesar de que no forma parte de un proceso de feature Engineering tradicional, comentaremos que es el **data-drift** y porque es importante que lo midas al momento de trabajar con tus variables.

## Escalamiento

Un paso importante antes de introducir features en los modelos, es el escalamiento de las variables. El objetivo de esto es que el dominio de las variables sea similar y de esta forma obtener mejores resultados. Este proceso es una de las cosas más sencillas que se pueden hacer y que (por lo general) se traduce en un aumento del rendimiento del modelo. Ojo que no hacer esto puede hacer que su modelo no tenga sentido como es el caso de algoritmos clásicos.

Tomar en consideración que muchos de los algoritmos modernos como **XGBoost** señalan que no necesitan escalamiento para el entrenamiento, sin embargo, escalar las variables de entrada puede impactar positivamente en el poder predictivo del modelo.

Para realizar el escalamiento utilizaremos el módulo `sklearn.preprocessing`, este entrega diversas técnicas de escalamiento, normalización y estandarización de datos a través de clases `Transformers` (no confundir con los transformers de Deep Learning).

### Estandarización

La estándarización es una de las transformaciones mas relevantes a tener presente durante el modelamiento. Esto debido a que un gran cantidad de algoritmos de aprendizaje automático / estadístico, asumen que los datos a operar se encuentran **distribuidos de manera normal**. **Si los datos no se distribuyen normalmente y contienen valores atípicos**, **es posible que la media y la desviación típica no reflejen con exactitud la tendencia central y la variabilidad de los datos**.

El objeto `StandarScaler` permite estandarizar datos para que tengan **media = 0 y desviación stándar = 1**. 

**Importante:** StandardScaler solo estandariza los datos, pero **no los normaliza!** Por lo tanto, sólo deben ocuparlo cuando:
- El algoritmo que ocupan requiere datos con distribución gaussiana
- Sus datos tienen distribución gaussiana.

**Ejemplo**

Comencemos graficando el histograma de algunas variables:

In [ ]:
import plotly.express as px

cols_to_estandarize = [
    'OverallQual',
    'OverallCond',
    'GarageCars',
    'GarageArea',
    'GrLivArea'
]
fig1 = px.histogram(df[cols_to_estandarize].melt(),
             x='value',
             color='variable',
             barmode='group')

fig1.show()

La transformación a aplicar es mover todos los datos a una distribución normal con media 0 y varianza 1.

Para esto, por cada dato:
    
$$ z = \frac{x - \mu}{\sigma}$$

En donde $\mu$ es la media de la columna y $\sigma$ es la desviación estándar.

Para esto, importamos escalador, lo inicializamos y luego ejecutamos `fit_transform` sobre los datos.
Notese que esto retorna un arreglo numpy con los datos escalados:

In [ ]:
data_to_standarize = df.loc[:, cols_to_estandarize] # separar data
data_to_standarize.head()

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

standard_scaler = StandardScaler() # inicializar scaler
standarized_df = standard_scaler.fit_transform(data_to_standarize) # escalar datos

standarized_df

Por comodidad, convertimos los datos escalados a un Dataframe:

In [ ]:
standarized_df = pd.DataFrame(standarized_df, columns=cols_to_estandarize)
standarized_df.head()

In [ ]:
standarized_df.describe()

Veamos ahora como se muestran las distribuciones de los datos estandarizados


Al aplicar `.fit_transform()` se obtienen los parámetros de media `.mean_` y desviación estándar `.scale_` para cada columna del dataframe operado. Observe que tales atributos del objeto tipo `StandardScaler` son públicos.

In [ ]:
standard_scaler

In [ ]:
standard_scaler.mean_

In [ ]:
standard_scaler.scale_

In [ ]:
px.histogram(standarized_df[cols_to_estandarize].melt(),
             x='value',
             color='variable',
             barmode='group')

In [ ]:
standarized_df.plot.kde(figsize=(16,9), bw_method=0.5)

> **Pregunta ❓**: ¿Qué sucede con las variables que se alejan mucho de ser normales?

In [ ]:
px.histogram(df, x='BsmtUnfSF')

### Escalamiento mínimo-máximo

Una buena alternativa al método anterior, es el escalamiento por rango, este tiene la forma:

\begin{equation}
\frac{x_{i} - \min(x)}{\max (x)-\min (x)}
\end{equation}

para $x$ columna a tratar, $x_i$ elemento a transformar. Esta transformación permite hacer que los datos se muevan entre 0 y 1 y puede ser utilizado sobre una distribución de datos no normales.

> **Nota:** este transformador se ve afectado por la **presencia de outliers**.

In [ ]:
cols_to_scale = ['BsmtUnfSF', 'Fireplaces']

In [ ]:
px.histogram(df, 'Fireplaces') # notar que solo tiene 3 valores posibles

In [ ]:
# BsmtUnfSF = Unfinished square feet of basement area
df.loc[:, 'BsmtUnfSF'].plot.hist(backend='plotly')

In [ ]:
data_to_scale = df.loc[:, cols_to_scale] # separar datos
data_to_scale

In [ ]:
from sklearn.preprocessing import MinMaxScaler

minmax_scaler = MinMaxScaler() # inicializar scaler

scaled_data = minmax_scaler.fit_transform(data_to_scale) # escalar datos
scaled_data = pd.DataFrame(scaled_data, columns=cols_to_scale) #  transformar a dataframe

scaled_data.head()

In [ ]:
scaled_data.describe()

In [ ]:
scaled_data.plot.hist(backend='plotly', barmode='overlay')

Comprobamos minimos y máximos:

In [ ]:
scaled_data.min()

In [ ]:
scaled_data.max()

**Nota**: Si se desea escalar por rango, la mejor práctica es comprender los mínimos y máximos *absolutos* para cada columna. Esto se refiere, a las cotas superiores e inferiores que posee la columna **por definición**. A modo de ejemplo, considere un dataframe con las notas de una asginatura donde se enzeña análisis de datos, se sabe que la nota máxima en cierto ítem se codifica en una columna y su máximo en efecto es 7.0. Sin embargo el mínimo en dicha columna es 1.5, que es distinto al mínimo natural para dicho item que es 1.0. Esto puede acarrear problemas con datos nuevos, sobretodo si aparece una nota inferior a 1.5.

### Outliers

Un outlier es un atípicos que están fuera del rango comun del resto de los datos.

In [ ]:
# histograma con los datos brutos
px.histogram(df, x='LotArea', marginal='box')

In [ ]:
# histograma con los datos escalados con MinMax
minmax_scaler = MinMaxScaler()
scaled_data = minmax_scaler.fit_transform(df.loc[:, ['LotArea']])
scaled_data = pd.DataFrame(scaled_data, columns=['LotArea'])
scaled_data.plot.hist(backend='plotly', barmode='overlay')

> **Pregunta:** ¿Qué pasa cuando queremos estandarizar pero tenemos outliers?

**Respuesta**: La gran mayoría de los datos tienen a quedar en un rango muy acotado. En el caso anterior, hacia la izquierda.

Hagamos el siguiente ejercicio: comparemos los estadísticos de los datos con y sin filtro IQR (datos contenidos en el rango intercuartílico):

In [ ]:
# media con los todos los datos de LotArea
original = round(df['LotArea'].describe(), 3)
original.name = 'LotArea Original'

# datos presentes solo en el rango intercuantílico
q1 = df['LotArea'].quantile(.25)
q3 = df['LotArea'].quantile(.75)
mask = df['LotArea'].between(q1, q3, inclusive='both')
iqr = df.loc[mask, 'LotArea']
iqr.name ='LotArea Filtro IQR'
iqr = round(iqr.describe(), 3)

Observen las diferencias entre las medias y las desviaciones estándar:

In [ ]:
desc = pd.concat([original, iqr], axis=1)
desc

Si normalizamos usando estandarización original, cada dato va a ser dividido por 9981, **9 veces más grande que la versión sin outliers usando IQR!**. Esto en términos prácticos hará que los datos tiendan a concentrarse mucho más **cercanos a la media**.

### Transformación Robusta

Cuando se trabaja con columnas que poseen valores fuera de rango (**outliers**) las transformaciones anteriores pueden fallar. En este caso, se recomienda utilizar una transformación similar a la estandarización, pero que trabaje sobre la mediana como media y el rango intercuantílico como desviación estándar:

\begin{equation}
\frac{x_i - Q_2(x)}{Q_3(x) - Q_1(x)}
\end{equation}

Donde $IQR = Q_3(x) - Q_1(x)$ es el rango intercuartílico de la columna $x$.

En otras palabras, por cada ejemplo se sustrae la mediana y se divide por el rango intercuartil 75%- 25%

In [ ]:
# Ejemplo para obtener IQR
data = np.array([1, 14, 19, 20, 22, 24, 26, 47])

# Calculamos rango intercurtil
q3, q1 = np.percentile(data, [75 ,25])
q3 - q1

**Ejemplo**

Se importa el objeto `RobustScaler` y se aplica

In [ ]:
px.histogram(df['LotArea'])

In [ ]:
from sklearn.preprocessing import RobustScaler

robust_scaler = RobustScaler()

lot_area_standarized = standard_scaler.fit_transform(df[['LotArea']])
robust_scaled_lot_area = robust_scaler.fit_transform(df[['LotArea']])

In [ ]:
comparacion = np.concatenate([lot_area_standarized, robust_scaled_lot_area], axis=1)
comparacion = pd.DataFrame(comparacion, columns=['Estandarizacion Común', 'Estandarización Robusta'])

In [ ]:
px.histogram(comparacion, marginal='box', barmode='overlay')

Podemos observar como los **datos en la estandarización robusta se distribuyen de forma mas amplia** que en caso de la estandarización normal.


### ¿Qué aspectos deberíamos considerar al momento de realizar escalamientos? 😅

Al momento de realizar escalamiento de cualquier tipo deben considerar los siguientes puntos:

- **El escalamiento es una fuente de data leakage** (Revisar más abajo).
- **Necesidad de entrenamiento ante data drift**. Muchos de los escalamientos necesitan el cálculo de un estadístico para realizar la transformación, esto implica que si los datos vistos durante el entrenamiento cambian en producción las transformaciones no serán las mejores y necesitarán un reentrenamiento de los datos.

---

## Codificación de Variables Ordinales

Para el manejo de adecuado de variables ordinales,  se recomienda expresar sus valores en función de códigos númericos. El transformer `OrdinalEncoder` permite transformar características categóricas en códigos enteros (números enteros) empezando desde 0 hasta N-1.

Veamos algunas variables ordinales:

BsmtQual: Evaluates the height of the basement

       Ex	Excellent (100+ inches)
       Gd	Good (90-99 inches)
       TA	Typical (80-89 inches)
       Fa	Fair (70-79 inches)
       Po	Poor (<70 inches
       NA	No Basement

BsmtCond: Evaluates the general condition of the basement

       Ex	Excellent
       Gd	Good
       TA	Typical - slight dampness allowed
       Fa	Fair - dampness or some cracking or settling
       Po	Poor - Severe cracking, settling, or wetness
       NA	No Basement

       
HeatingQC: Heating quality and condition

       Ex	Excellent
       Gd	Good
       TA	Average/Typical
       Fa	Fair
       Po	Poor

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()

variables_ordinal = df[['BsmtCond', 'BsmtQual','HeatingQC']] # separamos las variables ordinales

# Se entrena el codificador
ordinales = enc.fit_transform(variables_ordinal.dropna()) # se transforma variables a codificacion ordinal
ordinales

In [ ]:
pd.DataFrame(ordinales, columns = ['BsmtCond', 'BsmtQual','HeatingQC'])

Se obtienen las categorías

In [ ]:
enc.categories_ # verificamos el orden impuesto

> **Pregunta ❓**: ¿Existe algún problema en estas codificaciones?

In [ ]:
categories_order = [
    ["Po", "Fa", "TA", "Gd", "Ex"],
    ["Po", "Fa", "TA", "Gd", "Ex"],
    ["Po", "Fa", "TA", "Gd", "Ex"]
]

enc_2 = OrdinalEncoder(categories=categories_order)

ordinales = enc_2.fit_transform(variables_ordinal.dropna())

pd.DataFrame(ordinales, columns = ['BsmtCond', 'BsmtQual','HeatingQC'])

In [ ]:
enc_2.categories_

> **Pregunta ❓**: Codificamos estas variables con `variables_ordinal.dropna()`. ¿Por qué tuvimos que botar los valores faltantes y que se puede hacer en este caso ?

> **Pregunta ❓**: ¿Cómo codificamos las variables que no tienen orden?

## Codificación de Variables Categoricas

Un problema común con la códificación ordinal es que las variables pasan a ser consideradas continuas por algoritmos de machine learning (en especial por la API *estimators* de scikit-learn). Para evitar esto es posible convertir cada categoría en una columna por si sola y asignar un 1 cuando esté presente.

![One Hot Encoding](https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/14-Feature-Engineering-Parte-I/ohe.png?raw=true)
<center>Fuente: https://morioh.com/p/811a5d22bbca </center>

Esto se puede llevar a cabo por medio del transformador ` OneHotEncoder`.

In [ ]:
df['Foundation'].unique()

In [ ]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder() # inicializamos encoder

cod = ohe.fit_transform(df.loc[:, ['Foundation']]) # encodeamos categorias
cod # notar que es una matriz sparse

In [ ]:
ohe.categories_

### Cómo puedo transformar la matriz sparse?

#### Transformando a array

In [ ]:
cod.toarray()

In [ ]:
pd.DataFrame(cod.toarray(), columns=ohe.categories_) # generar dataframe con categorias onehot

#### O simplemente definiendo `sparse_output = False`

In [ ]:
ohe = OneHotEncoder(sparse_output = False) # inicializamos encoder
cod = ohe.fit_transform(df.loc[:, ['Foundation']]) # encodeamos categorias

cod

Volvamos a hacer lo mismo con `Neighborhood`

In [ ]:
df.loc[:, 'Neighborhood'].unique()

In [ ]:
# Ahora, con los barrios
ohe = OneHotEncoder(sparse_output = False) # inicializamos encoder con sparse_output = False
categories = ohe.fit_transform(df.loc[:, ['Neighborhood']]) # noten como ahora no uso .toarray()

pd.DataFrame(categories, columns = ohe.categories_)

## Pipelines

La manera anterior es bastante clara de comprender, sin embargo, es redundante y repite muchos patrones de asignación tediosos. Los `Pipelines` están diseñados para resolver este problema.


Las transformaciones en un dataset son combinadas entre si, hasta obtener una versión ordenada de los datos, posteriormente, estas se combinan con estimadores para formar un flujo de trabajo *input-output*. En Sckit-Learn el flujo antes nombrado de denomina *composite estimator* y se construye por medio de objetos tipo `Pipeline`.

Pero mas importante aun, los Pipelines nos ayudan resolver el problema de **Data Leakage** de forma natural.

Pero.. ¿Qué es **Data Leakage**?

### Data Leakage

Supongamos que queremos entrenar un modelo para predecir la variable `Accident`.

Para eso, un colega nos facilita código para pre procesar los datos y entrenar un modelo:

```python
def preprocess(df):

    """
    Prepara el dataframe para luego ser entrenado. En particular:
    - Imputa valores nulos
    - Genera features para aumentar la explicabilidad del modelo
    """

    df_proc = df.copy()

    # Imputar
    ## Weather
    weather_mode = df_proc["Weather"].mode().iloc[0]
    df_proc["Weather"] = df_proc["Weather"].fillna(weather_mode)

    ## Driver_Alcohol
    df_proc["Driver_Alcohol"] = df_proc["Driver_Alcohol"].fillna(0)

    ## Driver_Age
    age_mean = df_proc["Driver_Age"].mean()
    df_proc["Driver_Age"] = df_proc["Driver_Age"].fillna(age_mean)

    # Feature Engineering
    df_proc["Speed_Accident"] = df_proc["Speed_Limit"] * df_proc["Accident"]
    df_proc["Traffic_Norm"] = df_proc["Traffic_Density"] - df_proc["Traffic_Density"].mean()

    return df_proc

# aplicar preprocessing
df_proc = preprocess(df) 

# separar X,y
target = "Accident"
X = df.drop(columns = [target])
y = df[target]

# entrenar y evaluar performance
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, stratify=y)
model.fit(X_train, y_train)
```

> **Pregunta:** ¿Detectan algún problema en el código anterior? ¿Qué implicancias podría tener esto para el performance del modelo?

#### Definición 

**Data Leakage** (o fuga de datos) en Machine Learning ocurre cuando se **filtra información del conjunto de prueba** o de variables futuras hacia el modelo durante el entrenamiento. Esto provoca que el modelo aprenda patrones que no estarán disponibles en un entorno real, generando resultados demasiado optimistas en la validación y un desempeño pobre en producción.

#### ¿Por qué es importante hablar de Data Leakage?

El **Data Leakage** es uno de los errores más comunes —y a la vez más críticos— en los que puede incurrir un científico de datos. Su impacto suele pasar desapercibido durante el desarrollo del modelo, pero puede tener consecuencias graves una vez que se despliega en producción.

Ignorar o no detectar Data Leakage de forma oportuna puede acarrear los siguientes **riesgos**:

- **Sobreajuste (overfitting):** El modelo aprende patrones que en realidad no estarán disponibles en datos nuevos, lo que lleva a un desempeño artificialmente alto durante el entrenamiento pero pobre en producción.

- **Evaluaciones sesgadas:** Las métricas obtenidas durante la validación pueden ser optimistas o poco representativas, generando una falsa sensación de calidad del modelo.

- **Decisiones de negocio equivocadas:** Un modelo que parece funcionar bien en pruebas puede fallar en condiciones reales, provocando decisiones costosas para el negocio.

- **Pérdida de confianza en los modelos:** En industrias sensibles (salud, finanzas, etc), un modelo que falla por *data leakage* puede deteriorar la confianza de stakeholders y usuarios en la analítica predictiva.

### Como Definimos un Pipeline

Un pipeline es una **lista de tuplas**.

- La lista contiene todos los pasos que se efectuan desde la entrada hasta la salida del pipeline
- Cata tupla de la lista representa un subproceso del pipeline. Este debe estar compuesto por un nombre u la clase que corresponda.

![Pipeline](https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/15-Feature-Engineering-Parte-II/pipeline.png?raw=true)

Cada tupla debe seguir la siguiente estructura:

```python
step = ('alias', Transformer()) # Transformer = Procesamiento a realizar
```

Por ejemplo, para la imágen anterior, el pipeline sería:

```python
pipe = Pipeline([('scaling', Scaler()),
                 ('dimensionality_reduction', DimReductor()),
                 ('predictive_model', Model())])
```

Los **`pipelines`** nos ayuda en los siguientes puntos:

1. **Simplificación del proceso de aprendizaje automático**: Los Pipelines permiten <u>combinar múltiples pasos</u> en el proceso de aprendizaje automático, como el preprocesamiento de datos, la selección de características y el entrenamiento del modelo, en una única entidad. Esto simplifica el flujo de trabajo general y reduce las posibilidades de errores.

2. **Procesamiento de datos consistente**: Los Pipelines aseguran que los mismos pasos de preprocesamiento de datos se apliquen de manera consistente tanto a los datos de entrenamiento como a los de prueba. Esto reduce el riesgo de sobreajuste (producto de un data-leakage) y facilita la comparación del rendimiento de diferentes modelos.

3. **Ajuste de hiperparámetros fácil**: Los Pipelines permiten ajustar los hiperparámetros de varios pasos en el proceso de aprendizaje automático de manera simultánea. Esto puede ayudar a encontrar la combinación óptima de hiperparámetros y mejorar el rendimiento del modelo.

4. **Legibilidad y reutilización de código**: Los Pipelines proporcionan una forma clara y concisa de organizar el código, lo que facilita su lectura y mantenimiento. También permiten reutilizar la misma tubería para diferentes conjuntos de datos y modelos, ahorrando tiempo y esfuerzo.

5. **Mejora del rendimiento**: Al reducir la cantidad de manipulación de datos y cálculo requerido, los pipelines pueden llevar a tiempos de entrenamiento e inferencia de modelos más rápidos. Esto puede ser especialmente útil al trabajar con conjuntos de datos grandes o modelos complejos.

Veamos un ejemplo para transformar `LotArea` con `MinMaxScaler` y `StandardScaler`:

In [ ]:
from sklearn.pipeline import Pipeline

# instanciamos pipeline
LotArea_pipe = Pipeline([
    ('scaler_1', MinMaxScaler()),
    ('scaler_2', StandardScaler()),
    ])

# aplicamos pipeline sobre LotArea
result = LotArea_pipe.fit_transform(df[['LotArea']])

# generamos dataframe con resultado
pd.DataFrame(result, columns = LotArea_pipe.feature_names_in_)

### ColumnTransformer

Es muy frecuente, es que los datos sea heterogéneos por ejemplo, es normal encontrar datasets con variables **ordinales**, **categoricas**, y **numéricas**.

Para utilizar *pipelines* en este contexto, se necesitaria definir una por cada variable, repitiendo varios componentes de código entre variables que son del mismo tipo, esto resulta en una redundancia excesiva que se puede atacar por medio de objetos tipo `ColumnTransformer`. **Estos objetos permite separar flujos de preprocesamiento, permitiendo seleccionar por columna o grupos de columna dentro de un `pipeline`.**

Nuevamente, el ColumnTransformer se construye sobre una **lista de tuplas**:

- Cada tupla representa la transformación a aplicar sobre un conjunto de columnas
- Cada tupla debe contener el listado de columnas a transformar y la transformación a aplicar

Cada tupla debe seguir la siguiente estructura:

```python
transformation = ('alias', Transformer(), [col1, col2]) # Transformer = Procesamiento a realizar
```

Por ejemplo:

In [ ]:
from sklearn.compose import ColumnTransformer

preprocessing_transformer = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', OneHotEncoder(),  ['Neighborhood', 'Utilities', 'Foundation']),
        ('StandardScaler', StandardScaler(),['OverallQual', 'OverallCond', 'GarageCars', 'GarageArea']),
        ('PowerTransform', LotArea_pipe, ['LotArea'])]) # notar como se usa un pipeline dentro de ColumnTransformer

In [ ]:
# podemos encapsular el ColumnTransformer como paso de un Pipeline
housing_pipeline = Pipeline([
    ('Preprocessing', preprocessing_transformer)
])

Finalmente se aplican los procedimientos planificados en la variable `prep`

In [ ]:
df_preprocesado = housing_pipeline.fit_transform(df)
df_preprocesado

In [ ]:
df_preprocesado = pd.DataFrame(df_preprocesado.toarray())
df_preprocesado

> **Pregunta ❓**: ¿Qué sucedió con el resto de las columnas?

Nota que en este momento el Pipeline solo cuenta con una etapa de preprocesado. Sin embargo, la idea es que a futuro tenga el resto de los pasos de nuestro proyecto.


```python
housing_pipeline = Pipeline([
    ('Imputación', Imputador()),
    ('Preprocessing', Preprocesador()),
    ('Selector de Variables', SelectorDeVariables()),
    ('Reduccion de Dimensionalidad', ReduccionDeDimensionalidad()),
    ('Modelo', Modelo())

])
```

### En resumen... un Pipeline... 🧪


Un pipeline es una lista de tuplas.

- La lista contiene todos los pasos que se efectuan desde la entrada hasta la salida del pipeline
- Cata tupla de la lista representa un subproceso del pipeline. Este debe estar compuesto por un nombre u la clase que corresponda.

In [ ]:
num_cols = df.select_dtypes('float')
cat_cols = df.select_dtypes('object')

# Ejemplo de Preprocesador Compuesto
preprocessing = ColumnTransformer(
    transformers=[
        ("standard_scaler", StandardScaler(), num_cols),
        ("category_one_hot", OneHotEncoder(sparse_output=False, handle_unknown="ignore"), cat_cols),
    ]
)

## Manejo de valores faltantes

Por lo general, existen razones prácticas y conceptuales a tener en cuenta cuando se trabaja con valores faltantes.

### Conceptual

**La falta de información introduce sesgos en los modelos de datos**, pues hace que las muestras obtenidas no sean representativas del fenómeno que se desea estudiar. Esto puede generar conclusiones sesgadas y puede llevar a tomar malas decisiones.

### Práctica

**Los valores faltantes son incompatibles con algunos modelos de aprendizaje automático**, debido a que estos modelos son parte de la razón fundamental de analizar un fenómeno por medio de datos,es que se necesita comprender bien los mecanismos de manejo de este tipo de valores.

Veamos si nuestro dataset contiene valores nulos!

In [ ]:
# Print de valores nulos
df.isnull().sum().sort_values(ascending = False).iloc[:20]

## Tratamiento de Datos Faltantes: Deletion & Imputation ☠️

### Eliminación/Deletion 🪄

Es el método más sencillo, se conoce tambien como **list-wise deletion** y consiste en **eliminar filas o columnas de un dataset que presenten datos faltantes**. Se puede acceder a este tipo de tratamiento por medio de `.dropna()` objetos de Pandas.

Se recomiendan cuando el patrón de perdida de información observada (por ejemplo por medio de `mssingno`) es claramente aleatorio, y si además las variables con información faltante son 'pocas' y con 'pocos' valores faltantes. La definición de 'poco' varia en función del problema, pero una buena huerística puede ser inferior al 15% en variables de poca importancia. **Estos métodos generan una pérdida de datos y potencialmente aumento en el sesgo de los modelos** (👀 OJO: esto depende mucho del caso de estudio)

In [ ]:
df.dropna(subset=["LotFrontage"])

In [ ]:
df.drop(columns='Alley')

### Imputación 🧩

Corresponde a las técnicas que permiten rellenar de información faltante por medio de estimaciones.

> **Pregunta ❓:** ¿Se les ocurren ejemplos de casos de mala imputación?.

La imputación al igual que la eliminación de columnas y/o filas conlleva problemas. Entre los problemas de este método podemos encontrar: añadir ruido a nuestros datos, añadir sesgo y data leakage (¿Por qué?).

#### Imputación Simple

Por lo general este tipo de imputación presenta un buen rendimiento empírico en tareas de ciencia de datos y es ampliamente recomendado.

> **Nota:** aplicar este tipo de métodos puede afectar el calculo de varianzas y covarianzas.

En `pandas` podemos aplicar este tipo de imputación por medio del método `.fillna()` ya sea entregando un valor precalculado (media, mediana, moda, etc...) o utilizando los argumentos `ffill` (usar el valor anterior)  y `bfill` (usar el valor siguiente).
En `scikit-learn` por otra parte, podemos utilizar [`SimpleImputer`](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer), el cual es una transformación, lo que hace este método compatible con los `Pipelines`.

Existen directrices a tener en cuenta al momento de tratar valores faltantes:

**Variables categóricas**:

    * Transformar valores faltantes en una nueva categoría.
    * Utilizar códificación Dummy en variables categóricas

In [ ]:
df["GarageType"]

In [ ]:
df["GarageType"].value_counts(dropna=False)

In [ ]:
df[df["GarageType"].isna()]["GarageType"]

In [ ]:
garage_type = df["GarageType"].fillna("NoGarage")
garage_type.value_counts()

**Valores Ordinales**

    * Agregar la categoría de valor faltante como orden inicial o final en categorias ordinales.

In [ ]:
# ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
df["GarageCond"]

In [ ]:
df["GarageCond"].value_counts(dropna=False)

In [ ]:
garage_cond = df["GarageCond"].fillna("NoGarage")
# Agregar después al Encoder NoGarage ['NoGarage', 'Po', 'Fa', 'TA', 'Gd', 'Ex'],
garage_cond.value_counts()

**Variables numéricas**:

In [ ]:
df[df["LotFrontage"].isna()]

In [ ]:
df["LotFrontage"].describe()

En este caso, usaremos `SimpleImputer`.

La estrategia se define por el parámetro:

`strategy`, default=’mean’

    The imputation strategy.

        If “mean”, then replace missing values using the mean along each column. Can only be used with numeric data.

        If “median”, then replace missing values using the median along each column. Can only be used with numeric data.

        If “most_frequent”, then replace missing using the most frequent value along each column. Can be used with strings or numeric data. If there is more than one such value, only the smallest is returned.

        If “constant”, then replace missing values with fill_value. Can be used with strings or numeric data.


In [ ]:
from sklearn.impute import SimpleImputer

si = SimpleImputer(strategy="median")

imputed_lotfrontage = si.fit_transform(df.loc[:, ["LotFrontage"]])

imputed_lotfrontage = pd.DataFrame(imputed_lotfrontage, columns=["ImputedLotFrontage"])

imputed_lotfrontage

In [ ]:
imputed_lotfrontage.describe()

In [ ]:
import plotly.express as px

lf = pd.concat((df.loc[:, ["LotFrontage"]], imputed_lotfrontage))
px.histogram(lf, barmode="group")

#### Imputación Multivariada

Un problema de la imputación singular es que modela los datos como uno completo, sin considerar la incertidumbre inherente a todos los otros datos. Una solución para esto son los métodos de imputación múltiple.

Un método de inmputación multiple estima los valores faltantes a partir de otros. Puede ser tanto univariada (solo considerando la variable objetivo) como multivariada.

##### KNNImputer

Imputa usando k-Nearest Neighbors.

Los valores imputados se calculan según el parámetro `weights`:

- Si `weights=uniform`, se promedian los valores de los vecinos cercanos.
- Si `weights=distance`, se ponderan los valores de los vecinos cercanos según la distancia al punto.

<img src='https://github.com/MDS7202/MDS7202/blob/main/recursos/2023-01/15-Feature-Engineering-Parte-II/knn.png?raw=true' width=400/>

In [ ]:
import numpy as np

only_numeric = df.select_dtypes(include=np.number).drop(
    columns=[
        "YearBuilt",
        "YearRemodAdd",
        "YrSold",
        "GarageYrBlt",
        "SalePrice",
        "MoSold",
        "MSSubClass",
    ]
)
only_numeric

In [ ]:
from sklearn.impute import KNNImputer

# en este caso, simplemente se promedian los valores cercanos.
KNNimputer = KNNImputer(n_neighbors=2, weights="uniform") # notar como puedo definir el n° de vecinos
KNN_imputed_data = KNNimputer.fit_transform(only_numeric)
KNN_imputed_data = pd.DataFrame(KNN_imputed_data, columns=only_numeric.columns)

In [ ]:
lf_imputed = KNN_imputed_data.loc[:, ["LotFrontage"]]
lf_imputed.columns = ["LotFrontageImputed"]

lf = pd.concat((df.loc[:, ["LotFrontage"]], lf_imputed))
px.histogram(lf, barmode="group", nbins=50)

### Incluyendo la imputación en nuestra Pipeline

Pongamos en práctica todo lo que hemos aprendido en esta clase!

> **Ejercicio 📝**

0. Primero separemos la columna `SalePrice` del dataframe. Ejecute el siguiente código:

```python
target = 'SalePrice'
y = df[target].copy()
X = df.drop(columns = target).copy()
```

1. Usando el dataframe `X`, cree una lista conteniendo variables numéricas y otra lista conteniendo variables categóricas.
2. Para cada tipo de variable, genere un `pipeline`. En específico:
  - Numéricas:
    - Imputación multivariada con KNN
    - Escalamiento Robusto
  - Categóricas:
    - Imputación univariada (la que ustedes prefieran)
    - Transformación a One Hot
      - *Qué opción había que definir para evitar un "sparse output"?*
3. Con los pipelines creados, use un `ColumnTransformer` para englobarlos en un mismo paso.
4. Genere un `pipeline` que contenga como primer paso la transformación definida en el `ColumnTransformer`. Pruébelo sobre los datos e imprima la salida.
5. (Opcional) Vuelva a definir el `pipeline` anterior, pero definiendo como segundo paso el modelo de regresión que ustedes prefieran. Genere una predicción de  `SalePrice` y cuantifique el performance de su modelo usando [Mean Absolute Error](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_absolute_error.html#sklearn.metrics.mean_absolute_error).

**Respuesta:**

In [ ]:
# 0.
target = 'SalePrice'
y = df[target].copy()
X = df.drop(columns = target).copy()

In [ ]:
# 1.

In [ ]:
# 2.

In [ ]:
# 3.

In [ ]:
# 4.

In [ ]:
# 5.

## Ya pero quiero aprender mas... ¿algo para leer? 🤔

- Para comenzar pueden visualizar con mayor profundidad la documentación de Scikit-Learn enfocada en la extracción de features: https://scikit-learn.org/stable/modules/feature_extraction.html
- Complementar la clase leyendo el capitulo 5 del libro [Designing Machine Learning Systems](https://www.amazon.com/Designing-Machine-Learning-Systems-Production-Ready/dp/1098107969)

- Leer capitulo 1 del libro [Machine Learning Design Patterns: Solutions to Common Challenges in Data Preparation, Model Building, and MLOps](https://www.amazon.com/-/es/Valliappa-Lakshmanan/dp/1098115783/ref=pd_bxgy_img_sccl_1/142-9514380-0202369?pd_rd_w=JQSaa&content-id=amzn1.sym.26a5c67f-1a30-486b-bb90-b523ad38d5a0&pf_rd_p=26a5c67f-1a30-486b-bb90-b523ad38d5a0&pf_rd_r=BJE9WMJ9X1QQMYF4C3EJ&pd_rd_wg=y78Jr&pd_rd_r=6a1994c2-2734-428a-a270-ded23f892987&pd_rd_i=1098115783&psc=1)